In [6]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["NUM_THREADS"] = "4"
os.environ["OMP_NUM_THREADS"] = "4"
from keras.layers import Input, Dense, Flatten, Reshape
from sklearn.feature_extraction import image as sk_image
from concurrent.futures import ProcessPoolExecutor
import cartopy.feature as cfeature
from keras.models import Model
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from sklearn.cluster import AgglomerativeClustering
from scipy.signal import convolve2d 
from scipy import ndimage
from sklearn.cluster import KMeans
import numpy as np
import tensorflow as tf
from tensorflow import keras    
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from sklearn.cluster import DBSCAN
from pyhdf.SD import SD, SDC
import matplotlib as mpl
#tf.config.threading.set_inter_op_parallelism_threads(1)
from extract_training_data import *
from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d
from pyhdf.error import HDF4Error
from functions import *
from tensorflow.keras.models import load_model
from sklearn.cluster import KMeans, MiniBatchKMeans
import joblib
import plot_functions
import importlib 
importlib.reload(plot_functions)
from plot_functions import *
from sklearn.model_selection import train_test_split
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
# Visualize the result

In [8]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,"  , len(logical_gpus), "Logical GPUs")
        tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

import socket
hostname = socket.gethostname()
if "nird" in hostname:
    tf.config.threading.set_inter_op_parallelism_threads(8)
    data_loc = "/nird/projects/NS9600K/fslippe/data/"
    folder = "/nird/projects/NS9600K/data/modis/cao/MOD02/2020/ /nird/projects/NS9600K/data/modis/cao/MOD02/2021/ /nird/projects/NS9600K/data/modis/cao/MOD02/2023/"
if "mimi" in hostname:
    data_loc = "/uio/hume/student-u37/fslippe/data/"
    folder = "/scratch/fslippe/modis/MOD02/2019/ /scratch/fslippe/modis/MOD02/2020/ /scratch/fslippe/modis/MOD02/2021/ /scratch/fslippe/modis/MOD02/2022/ /scratch/fslippe/modis/MOD02/2023/ /scratch/fslippe/modis/MOD02/daytime_1km/ /scratch/fslippe/modis/MOD02/boundary_1km/ /scratch/fslippe/modis/MOD02/night_1km/ /scratch/fslippe/modis/MOD02/may-nov_2021/ /scratch/fslippe/modis/MOD02/cao_test_data/"


bands=[29]
#max_vals = np.load("%smodels/max_vals_dnb_l95_z50_ps128_(29)_cao_months_202012-202111.npy" %data_loc)
from autoencoder import SobelFilterLayer, SimpleAutoencoder
print(len(bands))

#encoder = load_model("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_band(6,20,29)_encoder")

1 Physical GPUs, 1 Logical GPUs
1


In [9]:
# Load training data 
patch_size = 128
model_run_name = "dnb_l95_z50_ps%s_band29" %(patch_size)

train_2018 = np.load("/scratch/fslippe/modis/MOD02/training_data/patch_size_%s/train_%s_%s.npy" %(patch_size, model_run_name, 2018))
train_2019 = np.load("/scratch/fslippe/modis/MOD02/training_data/patch_size_%s/train_%s_%s.npy" %(patch_size, model_run_name, 2019))
train_2020 = np.load("/scratch/fslippe/modis/MOD02/training_data/patch_size_%s/train_%s_%s.npy" %(patch_size, model_run_name, 2020))
train_2021 = np.load("/scratch/fslippe/modis/MOD02/training_data/patch_size_%s/train_%s_%s.npy" %(patch_size, model_run_name, 2021))
train_2022 = np.load("/scratch/fslippe/modis/MOD02/training_data/patch_size_%s/train_%s_%s.npy" %(patch_size, model_run_name, 2022))
train_2023 = np.load("/scratch/fslippe/modis/MOD02/training_data/patch_size_%s/train_%s_%s.npy" %(patch_size, model_run_name, 2023))

# Load test data 
test_2018 = np.load("/scratch/fslippe/modis/MOD02/training_data/patch_size_%s/test_%s_%s.npy" %(patch_size, model_run_name, 2018))
test_2019 = np.load("/scratch/fslippe/modis/MOD02/training_data/patch_size_%s/test_%s_%s.npy" %(patch_size, model_run_name, 2019))
test_2020 = np.load("/scratch/fslippe/modis/MOD02/training_data/patch_size_%s/test_%s_%s.npy" %(patch_size, model_run_name, 2020))
test_2021 = np.load("/scratch/fslippe/modis/MOD02/training_data/patch_size_%s/test_%s_%s.npy" %(patch_size, model_run_name, 2021))
test_2022 = np.load("/scratch/fslippe/modis/MOD02/training_data/patch_size_%s/test_%s_%s.npy" %(patch_size, model_run_name, 2022))
test_2023 = np.load("/scratch/fslippe/modis/MOD02/training_data/patch_size_%s/test_%s_%s.npy" %(patch_size, model_run_name, 2023))

# Combine all
combined_train_data = np.concatenate((train_2018, train_2019, train_2020, train_2021, train_2022, train_2023), axis=0)
combined_test_data = np.concatenate((test_2018, test_2019, test_2020, test_2021, test_2022, test_2023), axis=0)
max_val = np.max(combined_train_data)
min_val = np.min(combined_train_data) 
print("number of patches:", len(combined_train_data))
print("mean:", np.mean(combined_train_data))
print("max:", max_val)
print("min:", min_val)
normlized_train_data = (combined_train_data - min_val) / (max_val - min_val)
normlized_test_data = (combined_test_data - min_val) / (max_val - min_val)


FileNotFoundError: [Errno 2] No such file or directory: '/scratch/fslippe/modis/MOD02/training_data/patch_size_128/train_dnb_l95_z50_ps128_band29_2023.npy'

In [9]:
patches_per_file = 50000 / 128 * patch_size

for i in range(0, len(combined_train_data), patches_per_file):
    print(i)
    chunk = patches[i: i+patches_per_file]
    write_tfrecord(f'/scratch/fslippe/modis/MOD02/training_data/tf_data/normalized_trainingpatches_{model_run_name}_{i//patches_per_file}.tfrecord', chunk)

np.save("/uio/hume/student-u37/fslippe/data/models/max_vals_%s.npy" %(model_run_name), max_val)
np.save("/scratch/fslippe/modis/MOD02/training_data/tf_data/normalized_valpatches_%s" %(model_run_name), val_data)

0
50000
100000


NameError: name 'encoder' is not defined